In [1]:
import os, pandas, numpy as np
from amplpy import AMPL

In [4]:
# Initialize AMPL object
ampl = AMPL()

# Directory where the instance files are located
instance_dir = "instances"

# Ask user for the instance number to solve
instance = int(input("Enter the instance number to solve (1-21): "))
instance_file = os.path.join(instance_dir, f"inst{instance:02d}.dat")

# Read the first line of the instance file to get the value of n
with open(instance_file, 'r') as file:
    m = int(file.readline().strip())
    n = int(file.readline().strip())
    D = []
    l = np.array(file.readline().strip().split(), dtype=int)
    s = np.array(file.readline().strip().split(), dtype=int)


    for i in range(n+1):
        d_split = file.readline().split()
        d_row = []
        for j in d_split:
            d_row.append(int(j))
        D.append(d_row)
    Dnp = np.array(D)
        

# Check if the matrix is symmetric
is_symmetric = np.array_equal(Dnp, Dnp.T)

# After checking the symmetry on the original distance matrix (Dnp), it is processed to make it suitable for the
# adopted mip model. The final distance matrix has shape (n+m, n+2*m) and, to keep it consistent, the
# following adjustments are enforced:
# d_transformed[n+k,j] = Dnp[n+1,j] for all j, k;    d_transformed[i,n+m+k] = Dnp[i,0] for all i, k
# Make a copy of the distance matrix and adjust its shape
D_transformed = np.zeros((n + m, n + 2*m), dtype=int)

# Fill the transformed distance matrix
for i in range(n):
    
    for j in range(n):
        D_transformed[i, j] = Dnp[i, j]

for k in range(m):
    for j in range(n):
        D_transformed[n + k, j] = Dnp[n, j]

for i in range(n):
    for k in range(m):
        D_transformed[i, n + m + k] = Dnp[i, n]

# Create a copy of the data file with the transformed distance matrix
large_instance_file = os.path.join(instance_dir, f"inst{instance:02d}_large.dat")
with open(large_instance_file, 'w') as file:
    file.write(f"{m}\n")
    file.write(f"{n}\n")
    file.write(" ".join(map(str, l)) + "\n")
    file.write(" ".join(map(str, s)) + "\n")
    for row in D_transformed:
        file.write(" ".join(map(str, row)) + "\n")

# Load the AMPL model
# if is_symmetric:
#     ampl.read("model1_sb.mod")
# else:
#     ampl.read("model3.mod")
ampl.read("model3.mod")

#Starting from D compute Upper and Lower bound for the problem:
round_trips = []
for i in range(n):
    round_trips.append(Dnp[i][n] + Dnp[n][i])
LB = max(round_trips)
#LLB = min(round_trips)

UB = 0
for i in range(n):
    UB += max(Dnp[i, :])

ampl.get_parameter("LB").set(LB)
ampl.get_parameter("UB").set(UB)

# Load the data for the current instance
instance_file = os.path.join(instance_dir, f"inst{instance:02d}_transformed_large.dat")
ampl.readData(instance_file)

In [5]:
# Solve the model
ampl.setOption('randseed', 42) 
#ampl.solve(solver="cbc", cbc_options="seconds=300 timing=2 randomCbcSeed=42 randomSeed=42")
#ampl.solve(solver="highs", highs_options="timelim=300 timing=2")
ampl.solve(solver="gurobi", gurobi_options="timelim=300 timing=2 seed=42")
#ampl.solve(solver="xpress", xpress_options="timelim=300 timing=2")
#ampl.solve(solver="cplex", cplex_options="timelim=300 timing=2")
#ampl.solve(solver="copt", copt_options="timelim=300 timing=2")
#ampl.solve(solver="mosek", mosek_options="timelim=300 timing=2 seed=42")

# Get the solution fo'r the variables
x = ampl.getVariable("x").getValues()
delivery = ampl.getVariable("del").getValues()
maxDist = ampl.getVariable("maxDist").value()

# Print the results (or process them as needed)
print(f"\nResults for instance inst{instance:02d}.dat:")
print(f"x: {x}")
print(f"del: {delivery}")
print(f"maxDist: {maxDist}")
print("-" * 40)

Gurobi 11.0.2:   lim:time = 300
  tech:timing = 2
  tech:seed = 42
NL model read time = 0.500303s
NL model conversion time = 3.903779s
Gurobi 11.0.2: time limit, without a feasible solution
22559 simplex iterations
1 branching node
Setup time = 4.410574s
Solver time = 458.034778s
Output time = 0.002172s
Total time = 462.447524s

suffix time OUT;
suffix time_read OUT;
suffix time_setup OUT;
suffix time_output OUT;
suffix time_solver OUT;
suffix time_conversion OUT;

Results for instance inst12.dat:
x:    index0       index1    |    x.val    
     1              1       |      0      
     1              2       |      0      
     1              3       |      0      
     1              4       |      0      
     1              5       |      0      
     1              6       |      0      
     1              7       |      0      
     1              8       |      0      
     1              9       |      0      
     1              10      |      0      
     1              11 